In [75]:
#know we are just making a projct that pridect the performnaece of a student based on their study hours
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error,root_mean_squared_error
from sklearn.preprocessing import PolynomialFeatures

from sklearn.pipeline import make_pipeline

In [76]:
df=pd.read_csv('student_habits_performance.csv')

In [77]:
df.head()

,student_id,age,gender,study_hours_per_day,social_media_hours,netflix_hours,part_time_job,attendance_percentage,sleep_hours,diet_quality,exercise_frequency,parental_education_level,internet_quality,mental_health_rating,extracurricular_participation,exam_score
0,S1000,23,Female,0.0,1.2,1.1,No,85.0,8.0,Fair,6,Master,Average,8,Yes,56.2
1,S1001,20,Female,6.9,2.8,2.3,No,97.3,4.6,Good,6,High School,Average,8,No,100.0
2,S1002,21,Male,1.4,3.1,1.3,No,94.8,8.0,Poor,1,High School,Poor,1,No,34.3
3,S1003,23,Female,1.0,3.9,1.0,No,71.0,9.2,Poor,4,Master,Good,1,Yes,26.8
4,S1004,19,Female,5.0,4.4,0.5,No,90.9,4.9,Fair,3,Master,Good,1,No,66.4


In [78]:
df.shape


(1000, 16)

In [79]:
df.isna().sum()

student_id                        0
age                               0
gender                            0
study_hours_per_day               0
social_media_hours                0
netflix_hours                     0
part_time_job                     0
attendance_percentage             0
sleep_hours                       0
diet_quality                      0
exercise_frequency                0
parental_education_level         91
internet_quality                  0
mental_health_rating              0
extracurricular_participation     0
exam_score                        0
dtype: int64

In [80]:
df["gender"].value_counts()

gender
Female    481
Male      477
Other      42
Name: count, dtype: int64

In [81]:
df["gender1"]= df["gender"].map({'Male':0,'Female':1,'Other':2})
df["extracaliclam"]= df["extracurricular_participation"].map({'No':0,'Yes':1})
df["internet_qui"]= df["internet_quality"].map({'Poor':0,'Average':1,'Good':2})
df["par_time"]= df["part_time_job"].map({'No':0,'Yes':1})


In [82]:
df.head()

,student_id,age,gender,study_hours_per_day,social_media_hours,netflix_hours,part_time_job,attendance_percentage,sleep_hours,diet_quality,exercise_frequency,parental_education_level,internet_quality,mental_health_rating,extracurricular_participation,exam_score,gender1,extracaliclam,internet_qui,par_time
0,S1000,23,Female,0.0,1.2,1.1,No,85.0,8.0,Fair,6,Master,Average,8,Yes,56.2,1,1,1,0
1,S1001,20,Female,6.9,2.8,2.3,No,97.3,4.6,Good,6,High School,Average,8,No,100.0,1,0,1,0
2,S1002,21,Male,1.4,3.1,1.3,No,94.8,8.0,Poor,1,High School,Poor,1,No,34.3,0,0,0,0
3,S1003,23,Female,1.0,3.9,1.0,No,71.0,9.2,Poor,4,Master,Good,1,Yes,26.8,1,1,2,0
4,S1004,19,Female,5.0,4.4,0.5,No,90.9,4.9,Fair,3,Master,Good,1,No,66.4,1,0,2,0


In [83]:
cols = ['age','study_hours_per_day','social_media_hours','netflix_hours',
        'attendance_percentage','sleep_hours','exercise_frequency',
        'mental_health_rating','exam_score']

df_clean = df.copy()

for col in cols:
    Q1 = df_clean[col].quantile(0.25)
    Q3 = df_clean[col].quantile(0.75)
    IQR = Q3 - Q1

    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR

    df_clean = df_clean[(df_clean[col] >= lower) & (df_clean[col] <= upper)]


In [84]:
X = df.drop(['student_id','exam_score','parental_education_level','gender','part_time_job','diet_quality','parental_education_level','internet_quality','extracurricular_participation'], axis=1)
y = df['exam_score']
X.isnull().sum()


age                      0
study_hours_per_day      0
social_media_hours       0
netflix_hours            0
attendance_percentage    0
sleep_hours              0
exercise_frequency       0
mental_health_rating     0
gender1                  0
extracaliclam            0
internet_qui             0
par_time                 0
dtype: int64

In [98]:
X.head()

,age,study_hours_per_day,social_media_hours,netflix_hours,attendance_percentage,sleep_hours,exercise_frequency,mental_health_rating,gender1,extracaliclam,internet_qui,par_time
0,23,0.0,1.2,1.1,85.0,8.0,6,8,1,1,1,0
1,20,6.9,2.8,2.3,97.3,4.6,6,8,1,0,1,0
2,21,1.4,3.1,1.3,94.8,8.0,1,1,0,0,0,0
3,23,1.0,3.9,1.0,71.0,9.2,4,1,1,1,2,0
4,19,5.0,4.4,0.5,90.9,4.9,3,1,1,0,2,0


In [90]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = make_pipeline(PolynomialFeatures(degree=2), LinearRegression())
model.fit(X_train,y_train)
y_pred=model.predict(X_test)
mse=mean_squared_error(y_test,y_pred)
mae=mean_absolute_error(y_test,y_pred)
rmse=root_mean_squared_error(y_test,y_pred)         
r2=r2_score(y_test,y_pred)
print(f'Mean Squared Error: {mse}')
print(f'Mean Absolute Error: {mae}')
print(f'Root Mean Squared Error: {rmse}')
print(f'R^2 Score: {r2}')

Mean Squared Error: 24.8956037431609
Mean Absolute Error: 4.035428773733891
Root Mean Squared Error: 4.9895494529226685
R^2 Score: 0.9029142787167923


In [96]:
#checking for duplicates
df.duplicated().sum()


np.int64(0)

In [102]:
def predictions(input_data):
    input_df = pd.DataFrame([input_data], columns=X.columns)
    pred = model.predict(input_df)
    return pred[0]
predictions([21,1.4, 3.1, 1.3, 94.8, 8.0, 1, 1, 0, 0, 0, 0])

np.float64(38.95152281417914)